# Benchmarking Snapshotting / Altering the system and Running a Simulation

## First order of businees, you will need to have all the dependencies, the jupyter notebook is using.
* You can install a viable Virtulenvironment into the Jupyter kernel

## Boiler plate code:
#### I have made a simple interface, as I have thought that I will need to run many datapoints.

In [1]:
import pandas
from Utilities.SimulationInterface import SimulationInterface
from Utilities.Benchmarking import Benchmarking

#### Setup of a simple run with 10 000 particles
* I am setting the sigma here intentionally, as this is the variable controlling the ideal distance of molecules
* I am also setting the epsilon as this tells us the depth of the potential energy well

In [2]:
simulationObject = SimulationInterface(numberOfParticles = 10000, LJsigma = 0.001, LJepsilon = 0.001)
simulationObject.RunSetup()
simulationObject.RunSimulation(runLength=600)

HOOMD-blue 2.9.6 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 03/17/2021
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 10648 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10648
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:03 | Step 600 / 600 | TPS 170.917 | ETA 00:00:00
Average TPS: 170.859
---------
-- Neighborlist stats:
24 normal updates / 6 forced updates / 0 dangerous updates
n_neigh_mi

#### Collectring the data from the benchmarks into an empty Dictionary

In [3]:
benchmarkDictionary = {}

#### Benchmarking - Taking a Snapshot and restoring the hoomdSystemsObject

In [4]:
avgTime = Benchmarking.BenchmarkSnapshot(simulationObject.TakeSnapshotNRestore, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["SnapshotNRestore"] = [avgTime]

0. Run
1. Run
2. Run
3. Run
4. Run
5. Run
6. Run
7. Run
8. Run
9. Run
On average it took: 0.008913087844848632 seconds!


#### Benchmarking - Taking a Snapshot > Altering It and restoring the hoomdSystemsObject
* Interesingly, if you Snapshot > Alter > Restore many times, without running it for at least a step, hoomd Throws an error at you, that you have particles without position, even if you set them manually.

In [5]:
def benchmarkRunSnapshotNAlterNRun():
	simulationObject.TakeSnapshotAlterNRestore()
	simulationObject.RunSimulation(1)

avgTime = Benchmarking.BenchmarkSnapshot(benchmarkRunSnapshotNAlterNRun, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["AlterNRestore"] = [avgTime]

0. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10649
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:04 | Step 601 / 601 | TPS 13.5448 | ETA 00:00:00
Average TPS: 13.3529
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 36 / n_neigh_avg: 15.0308
shortest rebuild period: 100
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 3 / n_max: 15 / n_avg: 8.00075
** run complete **
1. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10650
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:04 | Step 602 / 602 | TPS 31.4386 | ETA 00:00:00
Average TPS: 30.4637
---------
-- Neighborlist stats:
0 normal updates / 0 force

#### Benchmarking Snapshot > Alter > Iterate > Restore

In [6]:
def benchmarkRunSnapshotNIterateNRun():
	simulationObject.TakeSnapshotIterateNRestore()
	simulationObject.RunSimulation(1)

avgTime = Benchmarking.BenchmarkSnapshot(benchmarkRunSnapshotNAlterNRun, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["IterateNRestore"] = [avgTime]

0. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10659
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:04 | Step 611 / 611 | TPS 34.3253 | ETA 00:00:00
Average TPS: 33.1466
---------
-- Neighborlist stats:
0 normal updates / 0 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 35 / n_neigh_avg: 15.0485
shortest rebuild period: 100
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 3 / n_max: 15 / n_avg: 8.00826
** run complete **
1. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10660
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:04 | Step 612 / 612 | TPS 33.5076 | ETA 00:00:00
Average TPS: 32.2425
---------
-- Neighborlist stats:
0 normal updates / 0 force

#### Benchmarking a 10 mins run (600 seconds, if a timestep is 1 second)

In [7]:
avgTime = Benchmarking.BenchmarkSimulation(simulationObject.RunSimulation, 600, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["Run10Mins"] = [avgTime]

** starting run **
Time 00:00:08 | Step 1220 / 1220 | TPS 205.83 | ETA 00:00:00
Average TPS: 205.687
---------
-- Neighborlist stats:
26 normal updates / 7 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 42 / n_neigh_avg: 15.1841
shortest rebuild period: 17
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 1 / n_max: 17 / n_avg: 8.01503
** run complete **
** starting run **
Time 00:00:11 | Step 1820 / 1820 | TPS 208.797 | ETA 00:00:00
Average TPS: 208.715
---------
-- Neighborlist stats:
27 normal updates / 7 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 45 / n_neigh_avg: 15.3121
shortest rebuild period: 17
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 1 / n_max: 18 / n_avg: 8.01503
** run complete **
** starting run **
Time 00:00:13 | Step 2420 / 2420 | TPS 217.124 | ETA 00:00:00
Average TPS: 217
---------
-- Neighborlist stats:
25 normal updates / 7 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 40 / n_neigh_avg: 1

### Showing the benchmarks next to each other.

In [8]:
BenchmarkRun = pandas.DataFrame.from_dict(benchmarkDictionary, orient="columns")
print(BenchmarkRun)

   SnapshotNRestore  AlterNRestore  IterateNRestore  Run10Mins
0          0.008913       0.060073         0.051229   2.829999
